# Mask R-CNN - Detection on Prostate Dataset


This notebook shows how to use trained Mask R-CNN on prostate dataset for a whole tile. As for large pathology image, we crop each image to several patches. This notebook is designed to get the detection reulst for single pic first and combine them back to the whole image. You'd need a GPU, though, because the network backbone is a Resnet101, which would be slow to detect on a CPU.

The code of the Prostate dataset can be found in prostate.py.

## Import Module

In [1]:
# import module from system lib
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import scipy.io

In [42]:
# import module from maskrcnn repo
from config import Config
import utils
import model as modellib
import visualize
from model import log
import prostate
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import compute_unary, create_pairwise_bilateral, \
     create_pairwise_gaussian, unary_from_softmax

%matplotlib inline

# Specify GPU to use
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

## Configurations

In [50]:
# Specify the dir that store the prostate dataset
# dataset_dir = os.path.join(os.path.dirname(os.getcwd()), "Data_Pre_Processing/cedars-224")
dataset_dir = os.path.join("/data/wenyuan/Mask-RCNN/Mask-RCNN-Path", "Data_Pre_Processing/cedars-224")
# We do 5-fold validation, specify which fold to be exclude for the current run
held_out_set = 4
# Featch the mean_pixel based on the training data (data exclude the held_out_set)
mean_pixel = prostate.Mean_pixel(dataset_dir, held_out_set)
# Configuration
class EvaluationConfig(prostate.ProstateConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.5
    DETECTION_NMS_THRESHOLD = 1     
    MEAN_PIXEL = np.array(mean_pixel)
    IMAGE_MAX_DIM = 512
    IMAGE_MIN_DIM = 512
    DETECTION_CROP = [128, 384, 128, 384] # [height_crop_start, height_crop_end, width_crop_start, width_crop_end]
    MODE = 16
    USE_TUMORCLASS = True
evaluation_config = EvaluationConfig()
evaluation_config.display()


Configurations:
BACKBONE_SHAPES                [[128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]
 [  8   8]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [ 0.1  0.1  0.2  0.2]
DETECTION_CROP                 [128, 384, 128, 384]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.5
DETECTION_NMS_THRESHOLD        1
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  512
IMAGE_MIN_DIM                  512
IMAGE_PADDING                  True
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [ 193.97800579  120.89113632  183.79060979]
MINI_MASK_SHAPE                (56, 56)
MODE                           16
NAME                           prostate
NUM_CLASSES       

## Create Model Graph and Loading Weights

In [51]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="detection", 
                          config=evaluation_config,
                          model_dir=MODEL_DIR)
# Get path to saved weights
# Either set a specific path, find a trained weights specified by epoch and held_out_set or find last trained weights
h5_filename = None # Specify the h5 filename here if you want to choose a specific file
epoch = 71

if h5_filename is not None:
    model_path = os.path.join(ROOT_DIR, ".h5 file name here")
elif epoch == -1:    
    model_path = model.find_last()[1]
else:
    try:
        model_path = model.find_specific(epoch = epoch, held_out_set = held_out_set)[1]
    except:
        model_path = model.find_specific(epoch = epoch)[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

Loading weights from  /scratch/wenyuan/Mask_RCNN_On_Pathology/Mask_RCNN/logs/prostate20180223T1554_held_out_set_4/mask_rcnn_prostate_0071.h5


## Prepare the Dataset

In [52]:
dataset_val = prostate.ProstateDataset()
_, val_list = dataset_val.generator_patition(dataset_dir, held_out_set)
# val_list = [image for image in val_list if image not in exclude_list]
dataset_val.load_prostate(dataset_dir, val_list, mode = evaluation_config.MODE)
dataset_val.prepare()

## Run Detection

In [53]:
def Post_Processing(img, prob):
    # change the prob (0, 0, 0, 0) item to (1, 0, 0 ,0)
    index_0, index_1 = np.where((prob == (0, 0, 0, 0)).all(axis = 2))
    prob[index_0, index_1, :] = (0.99, 0.01/3, 0.01/3, 0.01/3)
    # move the probability axis to assure that the first dimension is the class dimension
    prob_move = np.moveaxis(prob, 2, 0)
    # The input should be the negative of the logarithm of probability values
    # Look up the definition of the unary_from_softmax for more information
    unary = unary_from_softmax(prob_move)
    # The inputs should be C-continious -- we are using Cython wrapper
    unary = np.ascontiguousarray(unary)
    d = dcrf.DenseCRF(img.shape[0] * img.shape[1], 4)
    d.setUnaryEnergy(unary)
    # This potential penalizes small pieces of segmentation that are
    # spatially isolated -- enforces more spatially consistent segmentations
    feats = create_pairwise_gaussian(sdims=(7, 7), shape=img.shape[:2])
    d.addPairwiseEnergy(feats, compat=3,
                        kernel=dcrf.DIAG_KERNEL,
                        normalization=dcrf.NORMALIZE_SYMMETRIC)
    Q = d.inference(5)
    post_processing = np.argmax(Q, axis=0).reshape((img.shape[0], img.shape[1]))
    return post_processing

In [54]:
# Initialize the confusion matrix
C_MATRIX = np.zeros((4, 4))
# Create crop region
hv, wv = utils.create_crop_region(evaluation_config) # meshgrid for crop region
rc_num = int(math.sqrt(evaluation_config.MODE)) # how many patches in each row or col
# Process display setting
display_step = 10 # print out process for every display_step images
total_image = len(val_list)
r_tumor_probs = []
for Image_id in range(0, len(dataset_val.image_ids), evaluation_config.MODE):
# for Image_id in range(576, 577):
    image_whole = []
    gt_mask_whole = []
    det_mask_whole = []
    det_probs_whole = []
    for i in range(evaluation_config.MODE):
        # Load image and ground truth data
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_val, evaluation_config,
                                   Image_id + i, use_mini_mask=False)
        gt_tumor_class = 1 if (sum(gt_class_id)) else 0

        image_whole.append(image[hv, wv])
        # Convert gt-instance mask to gt-sementic mask
        gt_sementic_mask = utils.instance_2_sementic(gt_mask, gt_class_id)
        gt_sementic_mask = gt_sementic_mask['ATmask']
        gt_sementic_mask = gt_sementic_mask[hv, wv]
        gt_mask_whole.append(gt_sementic_mask)

        # Run object detection
        results = model.detect([image], verbose=0)
        r = results[0]

        # TODO: modify this part
        if np.argmax(r['tumor_probs']) == 1 and len(r['class_ids']) != 0:
            det_sementic_mask = r['sementic_mask']
            det_sementic_probs = r['prob_mask']
            r_tumor_probs.append(1)
        else:
            det_sementic_mask = np.zeros((image.shape[0], image.shape[1]))
            det_sementic_probs = np.zeros((image.shape[0], image.shape[1], evaluation_config.NUM_CLASSES))
            det_sementic_probs[:, :, 0] = 1
            r_tumor_probs.append(0)

        det_mask_whole.append(det_sementic_mask[hv, wv])
        det_probs_whole.append(det_sementic_probs[hv, wv])
    ## Combine Patches to Whole Slide
    img = utils.combine_2_whole_slide(image_whole, rc_num, rc_num)
    ann = utils.combine_2_whole_slide(gt_mask_whole, rc_num, rc_num)
    det = utils.combine_2_whole_slide(det_mask_whole, rc_num, rc_num)
    prob = utils.combine_2_whole_slide(det_probs_whole, rc_num, rc_num)
    post_processing = Post_Processing(img, prob)
    # Compute confusion matrix
    c_matrix = confusion_matrix(np.ravel(ann), np.ravel(post_processing))
    # Expand the c_matrix to NUM_CLASSES * NUM_CLASSES
    c_matrix = utils.expand_c_matrix(c_matrix, evaluation_config.NUM_CLASSES, ann, post_processing)
    # Update cofusion matrix
    C_MATRIX = C_MATRIX + c_matrix
    # Display the process
    if ((Image_id + 1) % (evaluation_config.MODE * display_step) == 0):
        print('Done evaluating %d / %d!'%((Image_id + 1) / evaluation_config.MODE, total_image))

## Print Evaluation Results

In [55]:
mIOU, IOU, below_th = utils.compute_mIOU(C_MATRIX, th = 0.5)
print('Confusion Matrix:\n', C_MATRIX)
print(' mIOU:', mIOU, '\n', 
      'IOU for each class:', IOU, '\n',
      'Below_th:', below_th)

Confusion Matrix:
 [[ 45358315.   1697425.   3663320.    858362.]
 [  1193875.  10930164.   1736414.    302055.]
 [  2681236.    584861.  31089459.     83417.]
 [   296534.    260562.    170666.   7096663.]]
 mIOU: 0.756885952125 
 IOU for each class: [0.81361567898526443, 0.65429099505571742, 0.77705439172965796, 0.78258274272933759] 
 Below_th: False


In [56]:
# import scipy.io
# filename = "Set%d_w_DNH_CRF"%held_out_set
# C_Matrix_dict = {filename:C_MATRIX}
# file_path = filename + ".mat"
# scipy.io.savemat(file_path, C_Matrix_dict)

In [57]:
len(r_tumor_probs)

1648

In [58]:
Tumor_indicator = []
for i in range(0, len(r_tumor_probs), evaluation_config.MODE):
    r = 1 if sum(r_tumor_probs[i : i + evaluation_config.MODE]) > 0 else 0
    Tumor_indicator.append(r)
    


In [59]:
np.savetxt("Set%d_ENH_RES.txt"%held_out_set, Tumor_indicator)

In [60]:
len(Tumor_indicator)

103